In [14]:
import pandas as pd
import requests
import numpy as np
import datetime

# Содержание
* [1. Данные по ипотечным ставкам](#chapter1)
* [2. Данные по ценам на недвижимость](#chapter2)
* [3. Макроэкономические данные](#chapter3)
* [4. Предобработка данных](#chapter4)

## 1. Данные по ипотечным ставкам <a class="anchor" id="chapter1"></a>

Источник: ЦБ РФ  
Период: 2013–2024 (11 лет)

- Ключевая ставка ЦБ РФ (%) - https://cbr.ru/hd_base/KeyRate/
- Cредний уровень процентной ставки по ипотечному кредиту, региональный разрез (%) - https://www.fedstat.ru/indicator/59319  
- Ставка по кредиту на квартиру с разбиением по типу ипотеки, при условиях (первоначальный взнос=30%, срок = 15 лет)  - https://дом.рф/analytics/mortgage/
- Cредневзвешенная процентная ставка по ипотечным жилищным кредитам в рублях, региональный разрез с разбиением по типу недвижимости, % - https://www.fedstat.ru/indicator/60293, https://cbr.ru/statistics/bank_sector/mortgage/?utm_source=w&utm_content=page
- Количество ипотечных жилищных кредитов, предоставленных физическим лицам-резидентам в рублях, региональный разрез  с разбиением по типу недвижимости, единиц - https://cbr.ru/statistics/bank_sector/mortgage/?utm_source=w&utm_content=page
- Объем жилищных кредитов в рублях, предоставленных физическим лицам-резидентам, региональный разрез  с разбиением по типу недвижимости, млн руб - https://cbr.ru/statistics/bank_sector/mortgage/?utm_source=w&utm_content=page


In [15]:
start_date =  datetime.date(2010, 1, 1).strftime('%d.%m.%Y')
end_date =  datetime.datetime.today().strftime('%d.%m.%Y')

# Ключевая ставка 
url_key_rate = f'https://cbr.ru/hd_base/KeyRate/?UniDbQuery.Posted=True&UniDbQuery.From={start_date}&UniDbQuery.To={end_date}'

response = requests.get(url_key_rate)
response.raise_for_status()

key_rate_df = pd.read_html(url_key_rate)[0]

key_rate_df.columns = ['date', 'key_rate']
key_rate_df['key_rate'] = key_rate_df['key_rate']/100

key_rate_df

,date,key_rate
0,17.03.2025,21.0
1,14.03.2025,21.0
2,13.03.2025,21.0
3,12.03.2025,21.0
4,11.03.2025,21.0
...,...,...
2872,23.09.2013,5.5
2873,20.09.2013,5.5
2874,19.09.2013,5.5
2875,18.09.2013,5.5


In [16]:
# Средний уровень процентной ставки по ипотечному кредиту, региональный разрез (%)

file_path = r'..\data\excel\home_average_mortgage_rate_region.xls'
home_mortgage_avg_rate_df = pd.read_excel(file_path, sheet_name='Данные', skiprows=2)
home_mortgage_avg_rate_df.columns = ['region','year', 'month', 'avg_rate_cbr']
home_mortgage_avg_rate_df['market_type'] = 'total'

home_mortgage_avg_rate_df


,region,year,month,avg_rate_cbr,market_type
0,Российская Федерация,2018,значение показателя за год,9.56,total
1,Российская Федерация,2019,январь,9.88,total
2,Российская Федерация,2019,февраль,10.04,total
3,Российская Федерация,2019,март,10.18,total
4,Российская Федерация,2019,апрель,10.28,total
...,...,...,...,...,...
7487,Чукотский автономный округ,2024,август,8.38,total
7488,Чукотский автономный округ,2024,сентябрь,8.57,total
7489,Чукотский автономный округ,2024,октябрь,8.86,total
7490,Чукотский автономный округ,2024,ноябрь,8.85,total


Ставка ЦБ РФ не совсем отражает реальность, поэтому используем дополнительный источник — данные от ДОМ.РФ

In [17]:
# Средний уровень процентной ставки по ипотечному кредиту, региональный разрез (доли)
# Динамика ставок предложения топ-20 ипотечных банков

file_path = r'..\data\excel\dom_rf_mortgage_rate.xlsx'
dom_rf_home_mortgage_avg_rate_df = pd.read_excel(file_path, skiprows=1)

dom_rf_home_mortgage_avg_rate_df = dom_rf_home_mortgage_avg_rate_df.drop('Unnamed: 0', axis=1)
dom_rf_home_mortgage_avg_rate_df = dom_rf_home_mortgage_avg_rate_df.drop('Unnamed: 2', axis=1)

collection_df_list = []

for i in range(0, len(dom_rf_home_mortgage_avg_rate_df)):   
    selected_rows = dom_rf_home_mortgage_avg_rate_df.iloc[i, :]
    selected_rows = selected_rows.transpose().reset_index()
    type_of_housing_market = selected_rows.iloc[0, 1]
    selected_rows = selected_rows.drop([0, 216])
    selected_rows['type_of_housing_market'] = type_of_housing_market
    selected_rows.columns = ['date', 'avg_rate_dom_rf', 'type_of_housing_market']
    collection_df_list.append(selected_rows)

dom_rf_home_mortgage_avg_rate_df = pd.concat(collection_df_list, axis=0, ignore_index=True)

dom_rf_home_mortgage_avg_rate_df

,date,avg_rate_dom_rf,type_of_housing_market
0,2020-12-25 00:00:00,0.0791,Новостройка
1,2021-01-08 00:00:00,0.0791,Новостройка
2,2021-01-15 00:00:00,0.079,Новостройка
3,2021-01-22 00:00:00,0.079,Новостройка
4,2021-01-29 00:00:00,0.0786,Новостройка
...,...,...,...
1070,2025-02-07 00:00:00,0.283199,Рефинансирование
1071,2025-02-14 00:00:00,0.2837,Рефинансирование
1072,2025-02-21 00:00:00,0.2837,Рефинансирование
1073,2025-02-28 00:00:00,0.2837,Рефинансирование


In [18]:
# Для обработки следующих excel таблиц
def preprocessing_region_excel_df(df: pd.DataFrame,name_column: str) -> pd.DataFrame:
    """
    Обрабатывает DataFrame, преобразуя его в длинный формат с добавлением столбца 'region'

    Параметры:
    - df: Исходный DataFrame.
    - name_column: Название столбца, которое будет использоваться для значений

    Возвращает:
    - Обработанный DataFrame
    """
    collection_df_list = []

    for i in range(0, len(df)):   
        selected_rows = df.iloc[i, :]
        selected_rows = selected_rows.transpose().reset_index()
        region = selected_rows.iloc[0, 1]
        selected_rows = selected_rows.drop(0)
        selected_rows['region'] = region
        selected_rows.columns = ['month', name_column, 'region']
        collection_df_list.append(selected_rows)

    df = pd.concat(collection_df_list, axis=0, ignore_index=True)
    return df



## 2. Макроэкономические данные <a class="anchor" id="chapter3"></a>

Источник: Росстат, ЦБ РФ  
Период: 2014-2024

- Уровень инфляции (%) - https://cbr.ru/statistics/ddkp/infl/?utm_source=w&utm_content=page
- Среднедушевые денежные доходы населения, региональный разрез (руб)
- Курс доллара (руб)


In [19]:
start_date =  datetime.date(2014, 1, 1).strftime('%d.%m.%Y')
end_date =  datetime.date(2024, 12, 31).strftime('%d.%m.%Y')

# Ключевая ставка + Инфляция

url = f'https://cbr.ru/statistics/ddkp/infl/?UniDbQuery.Posted=True&UniDbQuery.From={start_date}&UniDbQuery.To={end_date}'

response = requests.get(url)
response.raise_for_status()

key_rate_inflation_df = pd.read_html(url)[0]

key_rate_inflation_df.columns = ['date', 'key_rate', 'inflation', 'target_inflation']
key_rate_inflation_df['key_rate'] = key_rate_inflation_df['key_rate']/100
key_rate_inflation_df['inflation'] = key_rate_inflation_df['inflation']/100
key_rate_inflation_df = key_rate_inflation_df.drop('target_inflation', axis=1)

key_rate_inflation_df

,date,key_rate,inflation
0,12.2024,21.0,9.52
1,11.2024,21.0,8.88
2,10.2024,21.0,8.54
3,9.2024,19.0,8.63
4,8.2024,18.0,9.05
...,...,...,...
127,5.2014,7.5,7.59
128,4.2014,7.5,7.33
129,3.2014,7.0,6.92
130,2.2014,5.5,6.21


In [20]:
# Среднедушевые денежные доходы населения

file_path = r'..\data\excel\avg_monetary_income_people_region.xlsx'

avg_monetary_income_people_df = pd.read_excel(file_path, skiprows=2)
avg_monetary_income_people_df = avg_monetary_income_people_df.drop([97, 98, 99, 100, 101])

# Таблица представлена не очень удобном формате, поэтому обработаем ее
collection_df_list = []

for i in range(1, len(avg_monetary_income_people_df)):   
    selected_rows = avg_monetary_income_people_df.iloc[[0, i], :]
    selected_rows = selected_rows.transpose()
    region = selected_rows.iloc[0, 1]
    selected_rows = selected_rows.drop('Unnamed: 0')
    selected_rows = selected_rows.reset_index()
    selected_rows['region'] = region
    selected_rows.columns = ['year', 'quarter', 'income', 'region']
    collection_df_list.append(selected_rows)

avg_monetary_income_people_df = pd.concat(collection_df_list, axis=0, ignore_index=True)
avg_monetary_income_people_df['year'] = avg_monetary_income_people_df['year'].replace(r'Unnamed:*', pd.NA, regex=True)
avg_monetary_income_people_df['year'] = avg_monetary_income_people_df['year'].ffill()

avg_monetary_income_people_df.info()
avg_monetary_income_people_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6144 entries, 0 to 6143
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   year     6144 non-null   object
 1   quarter  6048 non-null   object
 2   income   6112 non-null   object
 3   region   6144 non-null   object
dtypes: object(4)
memory usage: 192.1+ KB


,year,quarter,income,region
0,2011\nгод,NaN,20771.642488,Российская Федерация
1,2012 год,I кв.,19105.67146,Российская Федерация
2,2012 год,II кв.,22572.96167,Российская Федерация
3,2012 год,III кв.,23262.024407,Российская Федерация
4,2012 год,IV кв.,27963.833212,Российская Федерация
...,...,...,...,...
6139,2023 год,IV кв. 2),156092,Чукотский авт.округ
6140,2023 год,год 3),156988,Чукотский авт.округ
6141,2024 год 2),I кв.,147290,Чукотский авт.округ
6142,2024 год 2),II кв.,157555,Чукотский авт.округ


In [21]:
# Курс доллара
start_date =  datetime.date(2014, 1, 1).strftime('%d/%m/%Y')
end_date =  datetime.date(2024, 12, 31).strftime('%d/%m/%Y')

xml_url = f'http://www.cbr.ru/scripts/XML_dynamic.asp?date_req1={start_date}&date_req2={end_date}&VAL_NM_RQ=R01235'

us_rate_df = pd.read_xml(xml_url)

us_rate_df

,Date,Id,Nominal,Value,VunitRate
0,01.01.2014,R01235,1,"32,6587","32,6587"
1,10.01.2014,R01235,1,"33,1547","33,1547"
2,11.01.2014,R01235,1,"33,2062","33,2062"
3,14.01.2014,R01235,1,"33,1204","33,1204"
4,15.01.2014,R01235,1,"33,2386","33,2386"
...,...,...,...,...,...
2710,25.12.2024,R01235,1,"99,8729","99,8729"
2711,26.12.2024,R01235,1,"99,6125","99,6125"
2712,27.12.2024,R01235,1,"99,2295","99,2295"
2713,28.12.2024,R01235,1,"100,5281","100,5281"


## 4. Предобработка данных <a class="anchor" id="chapter4"></a>

### Этапы предобработки данных:
- **Приведение столбцов с датами к единому формату**  
- **Удаление шума, исправление ошибок и устранение некорректных значений**  
- **Объединение данных из разных источников в единый набор**  
- **Преобразование данных в формат CSV для дальнейшей обработки**  

### Текущие наборы данных:
- **key_rate_df** — Ключевая ставка (%)
- **home_average_mortgage_rate_df** — Средний уровень процентной ставки по ипотечному кредиту, региональный разрез (%)
- **home_weight_avg_mortgage_rate_df** — Средневзвешенная процентная ставка по ипотечным кредитам, региональный разрез (%)
- **home_average_price_df** — Средняя цена 1 кв. м общей площади квартир на рынке жилья, региональный разрез (рубль)
- **home_index_price_df** — Индексы цен на рынке жилья, региональный разрез (%)
- **key_rate_inflation_df** — Инфляция (%)
- **avg_monetary_income_people_df** — Среднедушевые денежные доходы населения, региональный разрез (рубль)
- **us_rate_df** — Курс доллара, стоимость 1 доллара (рубль)



### 1) Работа со столбцом даты

- **Преобразуем столбец даты в формат `datetime`**
- **Убедимся, что все датасеты имеют единый диапазон дат**
- **Заполним пропущенные значения (NaN)**

In [22]:
# компоненты, необъодимые для предобработки
def datetime_year_month_df(df: pd.DataFrame) -> pd.DataFrame:
    """
    Преобразует DataFrame:
    - Удаляет строки с 'значение показателя за год'
    - Создаёт столбец 'date' в формате datetime64[ns] с использованием словаря месяцев
    - Удаляет столбцы 'year' и 'month'
    
    Args:
        df (pd.DataFrame): Исходный DataFrame с колонками 'year', 'month'
    
    Returns:
        pd.DataFrame: Обновлённый DataFrame с новым столбцом 'date'
    """

    month_dict = {
        'январь': '01', 'февраль': '02', 'март': '03', 'апрель': '04', 'май': '05',
        'июнь': '06', 'июль': '07', 'август': '08', 'сентябрь': '09', 'октябрь': '10',
        'ноябрь': '11', 'декабрь': '12'
    }
    df = df[df['month'] != 'значение показателя за год'].copy()
    df.loc[:, 'date'] = pd.to_datetime(df['year'].astype(str) + '-' + df['month'].map(month_dict) + '-01')
    df = df.drop(columns=['year', 'month'])

    return df

def datetime_year_quarter_df(df: pd.DataFrame) -> pd.DataFrame:
    """
    Преобразует DataFrame:
    - Создаёт столбец 'date' в формате datetime64[ns], используя year и quarter
    - Удаляет столбцы 'year' и 'quarter'
    
    Args:
        df (pd.DataFrame): Исходный DataFrame с колонками 'year', 'quarter'
    
    Returns:
        pd.DataFrame: Обновлённый DataFrame с новым столбцом 'date'
    """

    quarter_dict = {
        'I квартал': '01', 'II квартал': '04', 'III квартал': '07', 'IV квартал': '10'
    }
    df = df.copy()
    df.loc[:, 'date'] = pd.to_datetime(df['year'].astype(str) + '-' + df['quarter'].map(quarter_dict) + '-01')
    df = df.drop(columns=['year', 'quarter'])

    return df

# для понимания диапазона дат в каждом датасетие и выявления общего диапазона дат
min_date_list = [] # Список самой ранней даты по каждому набору
max_date_list = [] # Список самой поздней даты по каждому набору

In [23]:
# key_rate_df

# Создадим копию датафрейма для обработки, чтобы изменения не отражались на исходном наборе и мы могли откатится обратно
key_rate_df_copy = key_rate_df.copy()

key_rate_df_copy['date'] = pd.to_datetime(key_rate_df_copy['date'], format='%d.%m.%Y')

full_date_range = pd.date_range(start=key_rate_df_copy['date'].min(), end=key_rate_df_copy['date'].max(), freq='D')
full_date_df = pd.DataFrame(full_date_range, columns=['date'])

missing_dates = full_date_range.difference(key_rate_df_copy['date'])
print("Пропущенные даты:")
print(missing_dates)

Пропущенные даты:
DatetimeIndex(['2013-09-21', '2013-09-22', '2013-09-28', '2013-09-29',
               '2013-10-05', '2013-10-06', '2013-10-12', '2013-10-13',
               '2013-10-19', '2013-10-20',
               ...
               '2025-02-15', '2025-02-16', '2025-02-22', '2025-02-23',
               '2025-03-01', '2025-03-02', '2025-03-08', '2025-03-09',
               '2025-03-15', '2025-03-16'],
              dtype='datetime64[ns]', length=1323, freq=None)


Пропущенные даты - это выходные дни, заполним эти пропущенные ячейки значениями из соседних ячеек сверху

In [24]:
key_rate_df_copy = key_rate_df_copy.merge(full_date_df, how='outer', on='date')
key_rate_df_copy.isna().sum()

date           0
key_rate    1323
dtype: int64

Заполним значения `key_rate` в новых ячейках соседними значениями

In [25]:
key_rate_df_copy = key_rate_df_copy.ffill()
print(key_rate_df_copy.isna().sum())
key_rate_df_copy

date        0
key_rate    0
dtype: int64


,date,key_rate
0,2013-09-17,5.5
1,2013-09-18,5.5
2,2013-09-19,5.5
3,2013-09-20,5.5
4,2013-09-21,5.5
...,...,...
4195,2025-03-13,21.0
4196,2025-03-14,21.0
4197,2025-03-15,21.0
4198,2025-03-16,21.0
